____________
<b>This notebook reads .fchk files and extracts information about the atomic coordinates and vibrational modes.

The bottom cell will print out a string which you will need to copy and paste into the vibrations.js component on the website.

Just run the code cells below inputting the name of the molecule (as it appears in the folder name and file name) when prompted. </b>
___________

In [49]:
import csv
import pandas as pd

In [50]:
#These functions are work, but are written poorly.
#They could be improved if need be

def find_atomic_numbers(filepath):
    atomic_numbers = []
    with open(filepath, "r") as stream:
        file = list(stream)
    for i in range(0,len(file)):
        if file[i].startswith("Atomic numbers"):
            start_index = i+1
            break
    for i in range(0,len(file)):
        if file[i].startswith("Nuclear"):
            end_index = i-1
            break
    atomic_numbers_string = []
    for i in range(start_index, end_index+1):
        atomic_numbers_string.append(file[i])
    atomic_numbers = []
    for i in atomic_numbers_string:
        atomic_numbers.append([int(s) for s in i.split() if s.isdigit()])
    flat_atomic_numbers = [item for sublist in atomic_numbers for item in sublist]
    return flat_atomic_numbers

def find_coordinates(filepath):
    atomic_numbers = []
    with open(filepath, "r") as stream:
        file = list(stream)
    for i in range(0,len(file)):
        if file[i].startswith("Current"):
            start_index = i+1
            break
    for i in range(0,len(file)):
        if file[i].startswith("Force"):
            end_index = i-1
            break
    coordinates_string = []
    for i in range(start_index, end_index+1):
        coordinates_string.append(file[i])
    coordinates = []
    for i in coordinates_string:
        coordinates.append((i.split(" ")))
    filtered_coordinates = []
    for i in coordinates:
        temp = list((filter(None, i)))
        filtered_coordinates.append(temp)
    flat_coordinates = [float(item) for sublist in filtered_coordinates for item in sublist]
    num_atoms = int(len(flat_coordinates) / 3)
    final = []
    for i in range(0, num_atoms):
        start = i * 3
        final.append([flat_coordinates[start], flat_coordinates[start+1], flat_coordinates[start+2]])
    return final

def read_fchk(filepath):
    atomic_numbers = find_atomic_numbers(filepath)
    coordinates = find_coordinates(filepath)
    dataframe = pd.DataFrame(coordinates, index = range(0, len(atomic_numbers)))
    dataframe.columns=["x", "y", "z"]
    dataframe["atomic_number"] = atomic_numbers
    return (dataframe)

In [51]:
molecule = input("molecule name: ")
load_filepath = molecule + "/" + molecule + ".fchk"

molecule name: nh3


In [52]:
data = read_fchk(load_filepath)
data = data.reindex(sorted(data.columns), axis=1)
num_atoms = len(data)

__________________________________________________
<b>Run the cell below and copy what is printed out. Paste it into the vibrations.js component on a new line and it will be import into the ThreeD component automatically.

Again, this could probable be improved going forward so there is no need to copy and paste, but for now this works. </b>
__________________________________________________

In [53]:
json_string = data.transpose().to_json()
json_string = molecule + ": " + json_string
json_string = json_string[:-1] + ", num_atoms:" + str(num_atoms) + "},"
print(json_string)

nh3: {"0":{"atomic_number":7.0,"x":0.0,"y":0.0,"z":0.218982531},"1":{"atomic_number":1.0,"x":1.0500498e-16,"y":1.77534381,"z":-0.510959239},"2":{"atomic_number":1.0,"x":-1.53749284,"y":-0.887671905,"z":-0.510959239},"3":{"atomic_number":1.0,"x":1.53749284,"y":-0.887671905,"z":-0.510959239}, num_atoms:4},


In [42]:
num_atoms

4